In [8]:
import datasets
ds = datasets.load_from_disk("./taco_cleaned")
ds

Dataset({
    features: ['question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity'],
    num_rows: 9029
})

In [2]:
# figure out format of tests
import json
import sys
sys.set_int_max_str_digits(0)
keys = set()
types = set()
has_fn_name = 0

for ex in ds:
    eio = ex["input_output"]
    eio = json.loads(eio)
    types.add(type(eio))
    ks = eio.keys()
    if "fn_name" in ks:
        has_fn_name += 1
    for k in ks:
        keys.add(k)

print(types)
print(keys)
print(has_fn_name)

{<class 'dict'>}
{'fn_name', 'inputs', 'outputs'}
544


In [2]:
9000 * 5

# MinHash + Locality Sentitive Hasking, Jaccard Similarity 0.5
# pick 5 dedup
# 1 with original prompt, 4 LLM to rephrase the prompt

45000

In [10]:
ds = ds.shuffle()

In [11]:
import json
from tqdm import tqdm
from codeprm.execution import exec_io_test

subset = ds.select(range(10))
for i, ex in enumerate(subset):
    print(f"++ RUNNING {i}")
    for sol in tqdm(ex["solutions"][:5]):
        tests = json.loads(ex["input_output"])
        inputs = tests["inputs"]
        outputs = tests["outputs"]
        print(exec_io_test(sol, inputs, outputs))

++ RUNNING 0


 20%|███████████████████████████████████▊                                                                                                                                               | 1/5 [00:05<00:22,  5.56s/it]

(True, '')


 40%|███████████████████████████████████████████████████████████████████████▌                                                                                                           | 2/5 [00:11<00:16,  5.58s/it]

(True, '')


 40%|███████████████████████████████████████████████████████████████████████▌                                                                                                           | 2/5 [00:15<00:22,  7.62s/it]


KeyboardInterrupt: 

In [9]:
import json
only_fn_name = ds.filter(lambda ex: "fn_name" in json.loads(ex["input_output"]).keys())

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9029/9029 [00:04<00:00, 2053.23 examples/s]


In [3]:
ex = only_fn_name[1]
tests = json.loads(ex["input_output"])
print(tests["inputs"])

[['x - 5 = 20'], ['5 * x + 5 = 30'], ['20 = 5 * x - 5'], ['24 = 4 + 5 * x'], ['x = 5'], ['x * 100 = 700'], ['2 * x + 5 = 105'], ['2 * x = 198'], ['x - 100 + 2 - 50 = 52'], ['x / 3 = 33'], ['x + 80 = 20'], ['x + 20 = -60'], ['5 * x + 20 - x = 60'], ['x + x + 6 = 10'], ['5 * x = x + 8'], ['x = x / 2 + 25'], ['(5 - 3) * x = x + 2'], ['(x - 30) * 2 = x']]


In [13]:
import json
from tqdm import tqdm
from codeprm.execution import exec_named_test

subset2 = only_fn_name.select(range(10))
for i, ex in enumerate(subset2):
    print(f"++ RUNNING {i}")
    for sol in tqdm(ex["solutions"][:5]):
        tests = json.loads(ex["input_output"])
        inputs = tests["inputs"]
        outputs = tests["outputs"]
        name = tests["fn_name"]
        print(exec_named_test(sol, inputs, outputs, name))

++ RUNNING 0


 20%|███████████████████████████████████▊                                                                                                                                               | 1/5 [00:00<00:00,  4.55it/s]

(True, '')


 40%|███████████████████████████████████████████████████████████████████████▌                                                                                                           | 2/5 [00:00<00:00,  4.47it/s]

(True, '')


...

(True, '')


 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                   | 4/5 [00:06<00:01,  1.71s/it]

(True, '')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.70s/it]

(True, '')
